## Imports

In [1]:
#general imports
import pandas as pd
import numpy as np

#statsmodels for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

#scipy for testing
from scipy import stats

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime


## Bike Data

In [2]:
df_bikes = pd.read_csv('philadelphia_2017.csv')
df_weather = pd.read_csv('weather_hourly_philadelphia.csv')

df_bikes.head()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
0,1/1/2017 0:05,1/1/2017 0:16,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL"
1,1/1/2017 0:21,1/1/2017 0:57,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum
2,1/1/2017 0:22,1/1/2017 0:57,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum
3,1/1/2017 0:27,1/1/2017 0:39,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS"
4,1/1/2017 0:28,1/1/2017 0:36,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier


In [3]:
df_bikes.describe()

,start_station_id,end_station_id,bike_id
count,788907.000000,788907.000000,788907.000000
mean,3057.400081,3056.431487,5428.980913
std,41.126829,41.108792,3221.972215
min,3000.000000,3000.000000,0.000000
25%,3023.000000,3023.000000,3350.000000
50%,3050.000000,3049.000000,3703.000000
75%,3075.000000,3074.000000,5370.000000
max,3170.000000,3170.000000,11965.000000


## Weather Data

In [4]:
df_weather.head()

,date_time,max_temp,min_temp,precip
0,2015-01-02 01:00:00,3.3,3.3,0.0
1,2015-01-02 02:00:00,2.8,2.8,0.0
2,2015-01-02 03:00:00,2.2,2.2,0.0
3,2015-01-02 04:00:00,0.6,0.6,0.0
4,2015-01-02 05:00:00,1.1,1.1,0.0


In [5]:
df_weather.describe()

,max_temp,min_temp,precip
count,43785.000000,43785.000000,43785.000000
mean,14.204796,14.121300,0.087770
std,10.272673,10.259344,0.282963
min,-16.700000,-16.700000,0.000000
25%,6.100000,6.100000,0.000000
50%,15.000000,14.400000,0.000000
75%,22.800000,22.800000,0.000000
max,36.700000,36.700000,1.000000


In [6]:
"""
Die Funktion ist an sich jetzt richtig, die Wetterdaten müssen nur noch sortiert werden, hab ich jetzt leider nicht gerafft,
dass sie es nicht gemacht haben / die Daten extra kaputt gemacht haben :D
"""
def get_datetime(date_string):
    new_string = ""
    running_string = ""
    for char in date_string:
        if char == "/" or  char == ":" or char == " ":
            if len(running_string) == 1:
                running_string = "0" + running_string
            new_string += running_string + char
            running_string = ""
        else:
            running_string += char
    new_string += running_string
    if len(new_string) > 16:
        return datetime.strptime(new_string, '%m/%d/%Y %H:%M:%S')
    else:
        return datetime.strptime(new_string, '%m/%d/%Y %H:%M')

def would_be_rounded_up(num):
    if round(num) == int(num) + 1:
        return True
    else:
        return False

def get_temperature(start_time, end_time):
    duration_of_travel = end_time-start_time
    average_timestamp = end_time - datetime.fromisoformat("2017-01-01 00:00:00") - duration_of_travel/2
    weather_row = 17521 + average_timestamp.total_seconds()/3600
    as_int = int(weather_row)
    if would_be_rounded_up(weather_row):
        rows = [[df_weather["min_temp"][as_int],df_weather["max_temp"][as_int]], [df_weather["min_temp"][as_int+1],df_weather["max_temp"][as_int+1]]]
    else:
        rows = [[df_weather["min_temp"][as_int],df_weather["max_temp"][as_int]], [df_weather["min_temp"][as_int-1],df_weather["max_temp"][as_int-1]]]
    result = 0;
    avg_temp = (rows[0][0] + rows[0][1]) / 2
    result += avg_temp/(weather_row - int(weather_row))
    avg_temp = (rows[1][0] + rows[1][1]) / 2
    result += avg_temp/(1 - weather_row - int(weather_row))

    return result



def temperature_at_time(start_time, end_time):
    start_time = get_datetime(start_time)
    end_time = get_datetime(end_time)
    print(start_time)
    print(get_temperature(start_time, end_time))





temperature_at_time(df_bikes["start_time"][10000],df_bikes["end_time"][10000])



2017-01-13 06:34:00
16.243590901009362


In [7]:
unique_station_ids = set(df_bikes['start_station_id'])
id_mapped_to_station = []
for id in unique_station_ids:
    this_station = []
    this_station.append(id)
    this_station.append(df_bikes.loc[df_bikes['start_station_id'] == id].iloc[0]['start_station_name'])
    id_mapped_to_station.append(this_station)
print(id_mapped_to_station)


[[3072, 'Front & Carpenter'], [3073, '9th & Spring Garden'], [3074, '44th & Walnut'], [3075, 'Fairmount & Ridge'], [3077, '17th & Girard'], [3078, '19th & Market'], [3083, '3083'], [3086, 'Broad & Christian'], [3088, '3rd & Girard'], [3093, '33rd & Diamond'], [3095, '29th & Diamond'], [3096, '29th & Dauphin'], [3097, 'Berks Station, MFL'], [3098, '10th & Federal'], [3099, '4th & Washington'], [3100, 'Moyamensing & Tasker'], [3101, '11th & South'], [3102, 'Pennsylvania & Fairmount Perelman Building'], [3103, '27th & Master, Athletic Recreation Center'], [3104, '34th & Mantua'], [3105, 'Penn Treaty Park'], [3106, '33rd & Dauphin'], [3107, '33rd & Reservoir'], [3108, '15th & Market'], [3109, 'Parkside & Girard'], [3110, 'Del. River Trail & Penn St.'], [3111, 'Parkside & Belmont, Case Building'], [3112, '48th & Spruce'], [3113, 'Philadelphia Zoo'], [3114, '22nd & Federal'], [3115, '19th & Girard, PTTI'], [3116, 'Barnes Foundation'], [3117, 'ParkWest Town Center'], [3118, '46th Street Stati

In [9]:
# Zeiten in Daytime umgewandelt und Leihdauer hinzugefügt
df_bikes["start_time"] = pd.to_datetime(df_bikes["start_time"], dayfirst = True)

df_bikes["end_time"] = pd.to_datetime(df_bikes["end_time"], dayfirst = True)

df_bikes['duration'] = df_bikes['end_time'] - df_bikes['start_time']

df_bikes.head()


,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration
0,2017-01-01 00:05:00,2017-01-01 00:16:00,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL",0 days 00:11:00
1,2017-01-01 00:21:00,2017-01-01 00:57:00,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:36:00
2,2017-01-01 00:22:00,2017-01-01 00:57:00,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:35:00
3,2017-01-01 00:27:00,2017-01-01 00:39:00,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS",0 days 00:12:00
4,2017-01-01 00:28:00,2017-01-01 00:36:00,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier,0 days 00:08:00


In [10]:
df_bikes.notnull()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
788902,True,True,True,True,True,True,True,True,True
788903,True,True,True,True,True,True,True,True,True
788904,True,True,True,True,True,True,True,True,True
788905,True,True,True,True,True,True,True,True,True


In [11]:
df_bikes.dropna(axis=0,inplace=True)
len(df_bikes)-len(df_bikes.dropna())

0

In [13]:
# da hat sich jemand gegönnt :D
max(df_bikes['duration'])


Timedelta('91 days 16:29:00')

In [14]:
# auch nicht schlecht
min(df_bikes['duration'])


Timedelta('-323 days +00:14:00')

In [15]:
sumIndego30 = sum(1 for i in df_bikes["user_type"] if i == 'Indego30')
sumIndego30


667441

In [16]:
sumWalkUp = sum(1 for i in df_bikes["user_type"] if i == 'Walk-up')
sumWalkUp


100661

In [18]:
# gibt also noch mehr Modelle
sumIndego30 + sumWalkUp

768102

In [19]:
sumIndegoFlex = sum(1 for i in df_bikes["user_type"] if i == 'IndegoFlex')
sumIndegoFlex


6034

In [20]:
sumOneDayPass = sum(1 for i in df_bikes["user_type"] if i == 'One Day Pass')
sumOneDayPass


6328

In [21]:
sumIndego365 = sum(1 for i in df_bikes["user_type"] if i == 'Indego365')
sumIndego365


6840

In [23]:
#mindestends ein Modell fehlt noch
sumIndego30 + sumWalkUp + sumIndegoFlex + sumOneDayPass + sumIndego365


787304

In [24]:
##Wetter für 2017 in Philly
df_weather["date_time"] = pd.to_datetime(df_weather["date_time"])

start2017 = datetime(2017, 1, 1)
end2017 = datetime(2017, 12, 31)

df_2017weather = df_weather[(df_weather["date_time"] >= start2017) & (df_weather["date_time"] <= end2017)]
df_2017weather.tail()


,date_time,max_temp,min_temp,precip
26275,2017-12-30 20:00:00,-3.3,-3.3,0.0
26276,2017-12-30 21:00:00,-2.2,-2.2,0.0
26277,2017-12-30 22:00:00,-2.8,-2.8,0.0
26278,2017-12-30 23:00:00,-3.3,-3.3,0.0
26279,2017-12-31 00:00:00,-4.4,-4.4,0.0


In [25]:
df_2017weather.notnull()

,date_time,max_temp,min_temp,precip
17520,True,True,True,True
17521,True,True,True,True
17522,True,True,True,True
17523,True,True,True,True
17524,True,True,True,True
...,...,...,...,...
26275,True,True,True,True
26276,True,True,True,True
26277,True,True,True,True
26278,True,True,True,True


In [27]:
#ka warum der da kein Bock drauf hat
df_2017weather.dropna(axis=0,inplace=True)


<ipython-input-27-21f7db6c098e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017weather.dropna(axis=0,inplace=True)


In [28]:
# Werte für Temperaturen scheinen in Ordnung zu sein
max(df_weather["max_temp"])


36.7

In [29]:
min(df_weather["max_temp"])


-16.7